In [1]:
import bs4 as bs
import pandas as pd
import numpy as np
import requests
from time import sleep
from tqdm import tqdm
from user_agent import generate_user_agent
from pandas import ExcelWriter
import re
import multiprocessing as mp

# Rosagrotourism

In [ ]:
url_main = 'http://www.rosagrotourism.ru/cottages/1'
page_two = requests.get(url_main).text
soup_new = bs.BeautifulSoup(page_two, 'html.parser')

In [ ]:
soup_new

In [ ]:
soup_new.findAll("div", {"class" : 'domik-title content-p left'})[0].text.replace('\n', '')

In [ ]:
soup_new.findAll("div", { "class" : "cont-10" })[0].findAll("p", { "class" : "incont left"})[3].text

In [ ]:
soup_new.findAll("div", { "class" : "opisanie" })[1]

In [ ]:
url_head = 'http://www.rosagrotourism.ru/cottages?page='

In [ ]:
url_bas = 'http://www.rosagrotourism.ru/cottages/'
result = pd.DataFrame(columns=['Url', 'Name', 'Phone', 'Contact', 'EMail', 'Address', 'Description'])

In [ ]:
for i in tqdm(range(1, 398)):
    try:
        sleep(1)
        toappend = []
        toappend.append(url_bas+str(i))
        page_tmp = requests.get(url_bas+str(i), headers={'User-Agent': generate_user_agent()}).text
        soup_new = bs.BeautifulSoup(page_tmp, 'html.parser')
        contacts = soup_new.findAll("div", { "class" : "cont-10" })[0].findAll("p", { "class" : "incont left"})
        toappend.append(soup_new.findAll("div", {"class" : 'domik-title content-p left'})[0].text.replace('\n', ''))
        toappend.append(contacts[0].text[7:])
        toappend.append(contacts[1].text[15:])
        toappend.append(contacts[2].text[5:])
        toappend.append(contacts[3].text[5:])
        description = soup_new.findAll("div", { "class" : "opisanie" })
        toappend.append(" ".join([description[i].text.replace('\n', '') for i in range(len(description))]))
        result.loc[len(result)] = toappend
    except:
        print('Ошибка')
        

In [ ]:
result.to_excel("Parsing_glamping_res.xls", sheet_name="rosagrotourism")

# Glamping com

In [63]:
url_main = 'https://www.glamping.com/'
destinations = ['/destinations/africa/', '/destinations/asia/', '/destinations/europe/',
                '/destinations/australia/', '/destinations/north-america/',
                '/destinations/south-america/', '/destinations/europe/united-kingdom/',
                '/destinations/oceania/', '/destinations/central-america-and-caribbean/']

In [64]:
dest_page = requests.get('https://www.glamping.com/destinations/north-america/?page=1').text
soup = bs.BeautifulSoup(dest_page, 'html.parser')

In [65]:
soup.findAll("div", {"class": "pagination"})[0].findAll("a", {"class":"page"})[0].text

'2'

In [66]:
soup.findAll("a", {"class":"button destination_jump track-click"})[0]['href']

'/destination/north-america/michigan/harbert/the-snooty-fox-resort'

In [67]:
glamp_page = requests.get("https://www.glamping.com/destination/south-america/lima/san-jose-de-los-chorrillos/pariacaca-camp/").text
soup_new = bs.BeautifulSoup(glamp_page, "html.parser")

In [68]:
soup_new

<!DOCTYPE doctype html>

<html class="no-js" lang="en">
<head>
<meta charset="utf-8">
<!--  <meta name="viewport" content="width=device-width, initial-scale=1.0"> -->
<meta content="width=device-width, initial-scale=1.0,
minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">
<meta content="text/html; charset=utf-8" http-equiv="Content-type">
<meta content="" name="keywords">
<meta content="Pariacaca Camp is an ecolodge destination where you can enjoy tours and activities such as zip-lining, hiking, trekking, photography, swimming in the nearby waterfall and stargazing the area's clean and clear skies. The camp is all-inclusive, providing meals, guides and an accommodation inside one of our five tent domes with circular windows and a transparent top. Break the cycle of staying in the boring, traditional hotel and stay within nature in a different and friendly environment. Pariacaca Camp is comfortable, modern, eco-friendly and innovative. The entire camp can host up to

In [69]:
soup_new.findAll('div', {'class':'section_card title-card'})[0].findAll('h2')[0].text

'lima, peru'

In [70]:
glamping_com = pd.DataFrame(columns=['Url', 'Name', 'Placement'])

In [71]:
for dest in tqdm(destinations):
    sleep(1)
    url = url_main+dest
    dest_page = requests.get(url, headers={'User-Agent': generate_user_agent()}).text
    dest_soup = bs.BeautifulSoup(dest_page, 'html.parser')
    pages = dest_soup.findAll("div", {"class": "pagination"})[0].findAll("a", {"class":"page"})
    for page in pages:
        try:
            sleep(1)
            number = page.text
            url_page =url+'?page='+number
            page_for_soup = requests.get(url_page, headers={'User-Agent': generate_user_agent()}).text
            page_soup = bs.BeautifulSoup(page_for_soup, 'html.parser')
            glamps = page_soup.findAll("a", {"class":"button destination_jump track-click"})
        except Exception as ex:
            print(ex)
            print(url_page)
            continue
        for glamp in glamps:
            try:
                url_glamp = url_main+glamp['href']
                glamp_page = requests.get(url_glamp, headers={'User-Agent': generate_user_agent()}).text
                glamp_soup = bs.BeautifulSoup(glamp_page, 'html.parser')
                name = glamp_soup.findAll('div', {'class':'section_card title-card'})[0].findAll('h1')[0].text
                place = glamp_soup.findAll('div', {'class':'section_card title-card'})[0].findAll('h2')[0].text
                glamping_com.loc[len(glamping_com)] = [url_glamp, name, place]
            except Exception as ex:
                print(ex)
                print(url_glamp)

 22%|██▏       | 2/9 [07:35<28:12, 241.82s/it]

list index out of range
https://www.glamping.com//destination/europe/england/stroud/cotswold-bells


 67%|██████▋   | 6/9 [26:15<12:27, 249.30s/it]

list index out of range
https://www.glamping.com//destination/europe/england/stroud/cotswold-bells


100%|██████████| 9/9 [29:50<00:00, 139.10s/it]

list index out of range
https://www.glamping.com//destination/central-america-caribbean/puntarenas/uvita/rancho-pacifico


In [ ]:
glamping_com.head()

In [72]:
writer = ExcelWriter('Parsing_glamping_res_v_1.xls')
glamping_com.to_excel(writer, sheet_name="glampcom")
writer.save()

# Glamping hub

In [23]:
soup = bs.BeautifulSoup(requests.get('https://glampinghub.com').text, 'html.parser')
soup.findAll('ul', {'class':'dropdown-menu navbar-destinations-js'})[0]

<ul class="dropdown-menu navbar-destinations-js">
<li>
<a aria-expanded="false" aria-haspopup="true" class="dropdown-toggle" data-destination="destination-north-america-js" data-toggle="dropdown" href="#" role="button">
                  North America
                  <i class="caret caret-right"></i>
</a>
<ul class="dropdown-menu dropdown-submenu destination-north-america-js">
<li>
<a aria-expanded="false" aria-haspopup="true" class="dropdown-toggle" data-destination="destination-united-states-js" data-toggle="dropdown" href="#" role="button">
                      United States
                      <i class="caret caret-right"></i>
</a>
<ul class="dropdown-menu dropdown-submenu destination-united-states-js">
<li><a href="/unitedstatesofamerica/pacificwest/california/">
                        California
                      </a></li>
<li class="divider" role="separator"></li>
<li><a href="/unitedstatesofamerica/midatlantic/newyork/">
                        New York
              

In [56]:
all_links = [i for i in soup.findAll('ul', {'class':'dropdown-menu navbar-destinations-js'})[0].findAll('ul', {'class' : 'dropdown-submenu'})[0].findAll('ul', {'class':'dropdown-submenu'})[0].findAll('a')
             if 'View all' in i.text]

In [58]:
all_links[0]['href']

'/unitedstatesofamerica/'

In [62]:
main_soup = bs.BeautifulSoup(requests.get('https://glampinghub.com').text, 'html.parser')
all_links = []
for l1 in soup.findAll('a'):
    if 'View all' in l1.text:
        all_links.append(l1['href'])
all_links

['/unitedstatesofamerica/',
 '/canada/',
 '/mexico/',
 '/australia/newsouthwales/',
 '/australia/queensland/',
 '/australia/',
 '/france/',
 '/italy/',
 '/portugal/',
 '/spain/',
 '/unitedkingdom/',
 '/rentalsearch/europe-glamping-sites/',
 '/chile/',
 '/brazil/',
 '/ecuador/',
 '/argentina/',
 '/rentalsearch/southamerica-accommodations-glamping/',
 '/southafrica/',
 '/kenya/',
 '/rentalsearch/africa-safari-accommodations/',
 '/thailand/',
 '/srilanka/',
 '/india/',
 '/rentalsearch/asia-accommodations-glamping/']

In [74]:
page_ex = bs.BeautifulSoup(requests.get('https://glampinghub.com/unitedstatesofamerica/').text, 'html.parser')
page_ex.findAll('p', {'class':'spaceout-0 spacein-rl-10 spacein-tb-5'})

[<p class="spaceout-0 spacein-rl-10 spacein-tb-5"><span class="font-weight-bolder">1-30 of 5545 accommodations</span> in the hub
           </p>, <p class="spaceout-0 spacein-rl-10 spacein-tb-5">
 <strong>1-30</strong> of
             <span class="font-weight-bolder">5545 accommodations</span> in the hub for United States of America
           
         </p>]

In [107]:
page_ex.findAll('span', {'class':'font-weight-bolder'})[0].text.split()[-2]

'5546'

In [123]:
page_glamp = requests.get('https://glampinghub.com/italy/tuscany/bucine/glamping-tuscany-italy/').text
last_soup = bs.BeautifulSoup(page_glamp, 'html.parser')

In [127]:
last_soup.findAll('a', {'class':'accommodation-location_link link link-secondary spaceout-r-5'})

[<a class="accommodation-location_link link link-secondary spaceout-r-5" href="#location">
 <i class="fa fa-map-marker"></i> Bucine, Tuscany (Italy)
         </a>]

In [99]:
glamphub = pd.DataFrame(columns=['Url', 'Name', 'Place', 'Description'])
for link in tqdm(all_links):
    sleep(1)
    main_url = 'https://glampinghub.com'+link
    page_ex = bs.BeautifulSoup(requests.get(main_url, headers={'User-Agent': generate_user_agent()}).text, 'html.parser')
    low_bound = 1
    high_bound = int(page_ex.findAll('span', {'class':'font-weight-bolder'})[0].text.split()[-2])/30+1
    for i in range(low_bound, high_bound+1):
        sleep(1)
        try:
            page_url = main_url+"?page="+str(i)
            page_wrk = bs.BeautifulSoup(requests.get(page_url, headers={'User-Agent': generate_user_agent()}).text, 'html.parser')
            glamp_arr = page_wrk.findAll('a', {'class':'request-booking'})
        except Exception as ex:
            print(ex)
            print(page_url)
            continue
        for glamp in glamp_arr:
            try:
                glamp_url = 'https://glampinghub.com'+glamp['href']
                glamp_page = requests.get(glamp_url, headers={'User-Agent': generate_user_agent()}).text
                glamp_soup = bs.BeautifulSoup(glamp_page, 'html.parser')
                glamphub.loc[len(glamphub)] = [glamp_url, glamp_soup.findAll('h1', {'class':'secondary-textColor'})[0].text,
                                              glamp_soup.findAll('a', {'class':'accommodation-location_link link link-secondary spaceout-r-5'})[0].text.replace('\n',''),
                                              glamp_soup.findAll('p', {'class':'partial-collapse-active-js'})[0].text.replace('\n', '')]
            except Exception as ex:
                print(ex)
                print(glamp_url)


  0%|          | 0/24 [00:00<?, ?it/s]


list index out of range
https://glampinghub.com/unitedstatesofamerica/mountainstates/colorado/westcliffe/luxury-tents-near-colorado-springs/
list index out of range
https://glampinghub.com/unitedstatesofamerica/southwest/colorado/denver/airstream-rental-colorado/
list index out of range
https://glampinghub.com/unitedstatesofamerica/mountainstates/wyoming/wilson/cabins-unique/
list index out of range
https://glampinghub.com/unitedstatesofamerica/midwest/missouri/gallatin/log-cabin-rentals-northern-missouri/


KeyboardInterrupt: 

In [131]:
def worker(in_queue, out_queue):
    while not in_queue.empty():
        link = in_queue.get(False)
        glamphub = pd.DataFrame(columns=['Url', 'Name', 'Place', 'Description'])
        main_url = 'https://glampinghub.com'+link
        page_ex = bs.BeautifulSoup(requests.get(main_url, headers={'User-Agent': generate_user_agent()}).text, 'html.parser')
        low_bound = 1
        high_bound = int(int(page_ex.findAll('span', {'class':'font-weight-bolder'})[0].text.split()[-2])/30+1)
        for i in range(low_bound, high_bound+1):
            try:
                page_url = main_url+"?page="+str(i)
                page_wrk = bs.BeautifulSoup(requests.get(page_url, headers={'User-Agent': generate_user_agent()}).text, 'html.parser')
                glamp_arr = page_wrk.findAll('a', {'class':'request-booking'})
            except Exception as ex:
                print(ex)
                print(page_url)
                continue
            for glamp in glamp_arr:
                try:
                    glamp_url = 'https://glampinghub.com'+glamp['href']
                    glamp_page = requests.get(glamp_url, headers={'User-Agent': generate_user_agent()}).text
                    glamp_soup = bs.BeautifulSoup(glamp_page, 'html.parser')
                    toappend = [glamp_url, glamp_soup.findAll('h1', {'class':'secondary-textColor'})[0].text,
                                                  glamp_soup.findAll('a', {'class':'accommodation-location_link link link-secondary spaceout-r-5'})[0].text.replace('\n','')]
                    if len(glamp_soup.findAll('p', {'class':'partial-collapse-active-js'})) > 0:
                        toappend.append(glamp_soup.findAll('p', {'class':'partial-collapse-active-js'})[0].text.replace('\n', ''))
                    else:
                        toappend.append("Not parsed")
                    glamphub.loc[len(glamphub)] = toappend
                except Exception as ex:
                    print(ex)
                    print(glamp_url)
        out_queue.put(glamphub)

In [132]:
in_queue = mp.Manager().Queue()
out_queue = mp.Manager().Queue()
n_process = 10

In [133]:
for link in tqdm(all_links):
    in_queue.put(link)
    
jobs = []
for i in range(n_process):
    job = mp.Process(target=worker, args=(in_queue, out_queue, ))
    job.start()
    jobs.append(job)
    
for job in jobs:
    job.join()

result = pd.DataFrame(columns=['Url', 'Name', 'Place', 'Description'])
while not out_queue.empty():
    if len(result)==0:
        result = out_queue.get()
    else:
        result = pd.concat([result, out_queue.get()])

result.head()


100%|██████████| 24/24 [00:00<00:00, 2333.90it/s]











,Url,Name,Place,Description
0,https://glampinghub.com/australia/queensland/g...,Gorgeous Glass Cabin Rentals in Bushlands of Q...,"Glass House Mountains, Queensland (Australia)...","Each of these glass cabin rentals, l..."
1,https://glampinghub.com/australia/queensland/m...,"Secluded Cozy Cottage in Mount Glorious, Austr...","Mount Glorious, Queensland (Australia)",Ideal for a romantic escape or a fam...
2,https://glampinghub.com/australia/queensland/m...,"Romantic Cottage with Veranda in Rainforest, A...","Mount Glorious, Queensland (Australia)","Just 40 minutes drive from Brisbane,..."
3,https://glampinghub.com/australia/queensland/m...,Luxurious Beachfront Huts with Pool in Queensl...,"Mission Beach, Queensland (Australia)","These luxury huts in Queensland, Aus..."
4,https://glampinghub.com/australia/queensland/d...,Family-Friendly Luxury Safari Tent Rentals on ...,"Diddillibah, Queensland (Australia)",Escape to this memorable and divine ...


In [134]:
len(result)

9996

In [135]:
writer = ExcelWriter('Parsing_glamping_res_v_11.xls')
result.to_excel(writer, sheet_name="glamphub")
writer.save()

# Last site

In [189]:
page_accomod_ex = bs.BeautifulSoup(requests.get('http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11344').text, 'html.parser')

In [195]:
page_accomod_ex.findAll('h1')[0].text.encode('ISO-8859-1').decode('utf-8', 'ignore')

'Отель «Soldaya Grand Hotel & Resort»'

In [210]:
page_accomod_ex.findAll('div', {'class':'detail-field'})[1].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]

'     \ufeff  Гостиницы и иные средства размещения'

In [211]:
page_accomod_ex.findAll('div', {'class':'detail-field'})[4].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]

'ООО «Туристическая база «Восход»'

In [212]:
page_accomod_ex.findAll('div', {'class':'detail-field'})[5].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]

'Республика Крым'

In [213]:
page_accomod_ex.findAll('div', {'class':'detail-field'})[8].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]

'298000, Россия, Республика Крым, г. Судак, ул. Морская, д. 23'

In [214]:
page_accomod_ex.findAll('div', {'class':'detail-field'})[9].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]

'+ 7 (978) 832-17-34'

In [215]:
page_accomod_ex.findAll('div', {'class':'detail-field'})[11].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]

'info@soldayahotel.com'

In [216]:
page_accomod_ex.findAll('div', {'class':'detail-field'})[12].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]

'http://www.soldayahotel.com/'

In [2]:
def worker(in_queue, out_queue):
    while not in_queue.empty():
        
        url = in_queue.get(False)
        try:
            page_accomod = bs.BeautifulSoup(requests.get(url).text, 'html.parser')
        except Exception as ex:
            continue
        try:
            toappend=[]
            toappend.append(url)
            toappend.append(page_accomod.findAll('h1')[0].text.encode('ISO-8859-1').decode('utf-8', 'ignore'))
            typeField = page_accomod.findAll('div', {'class':'detail-field'})[1].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]
            toappend.append(typeField)
            ownerField = page_accomod.findAll('div', {'class':'detail-field'})[4].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]
            toappend.append(ownerField)
            regionField = page_accomod.findAll('div', {'class':'detail-field'})[5].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]
            toappend.append(regionField)
            adressField = page_accomod.findAll('div', {'class':'detail-field'})[8].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]
            toappend.append(adressField)
            phoneField = page_accomod.findAll('div', {'class':'detail-field'})[9].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]
            toappend.append(phoneField)
            emailField = page_accomod.findAll('div', {'class':'detail-field'})[11].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]
            toappend.append(emailField)
            siteField = page_accomod.findAll('div', {'class':'detail-field'})[12].text.encode('ISO-8859-1').decode('utf-8', 'ignore').split('\n')[-2]
            toappend.append(siteField)
            out_queue.put(toappend)
        except Exception as ex:
            print(ex)
            print(url)
            continue

In [3]:
in_queue = mp.Manager().Queue()
out_queue = mp.Manager().Queue()
n_process = 5

In [4]:
for i in range(1, 5000):
    in_queue.put('http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/'+str(i))
    
jobs = []
for i in range(n_process):
    job = mp.Process(target=worker, args=(in_queue, out_queue, ))
    job.start()
    jobs.append(job)
    
for job in jobs:
    job.join()

result = pd.DataFrame(columns=['Url', 'Name', 'Type', 'Owner', 'Region', 'Adress', 'Phone', 'Email', 'Site'])
while not out_queue.empty():
    result.loc[len(result)] = out_queue.get()

result.head()

list index out of range
list index out of range
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/3
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/2
list index out of range
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/1
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/4
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/5
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/9
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/7
list index out of range
list index out of range
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/8
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/10
http://xn----7sba3acabbldhv3ch

Process Process-5:
Traceback (most recent call last):
  File "/home/kinetik/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/kinetik/anaconda3/envs/py35/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-4a9fab34080d>", line 4, in worker
    url = in_queue.get(False)
  File "<string>", line 2, in get
  File "/home/kinetik/anaconda3/envs/py35/lib/python3.5/multiprocessing/managers.py", line 732, in _callmethod
    raise convert_to_error(kind, result)
queue.Empty


,Url,Name,Type,Owner,Region,Adress,Phone,Email,Site
0,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"Отель ""Радужный"" ИП Дьяченко А.А.",Тип:,Наименование собственника:,Краснодарский край,"354008, Краснодасркий край, г. Сочи, ул. Пирог...",Телефон:,E-mail:,Адрес сайта:
1,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,Общество с ограниченной ответственностью «Сан...,"﻿ Дома отдыха, пансионаты и другие анало...",Общество с ограниченной ответственностью «Сан...,Краснодарский край,"354019, г. Сочи, Курортный проспект, д. 87",(8622) 67-16-70,frunze_sochi@mail.ru,http://www.frunze.net/
2,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ЗАО ""Нева"" Пансионат ""НЕВА ИНТЕРНЕЙШЕНЕЛ"" дире...",Тип:,Наименование собственника:,Краснодарский край,"г.Сочи , ул.Виноградная, 101 т.53-62-39,ф.53-5...",Телефон:,E-mail:,Адрес сайта:
3,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ООО""ВИЛ"",""Шале Поляна"" директор Шпалов Юрий Ми...",Тип:,Наименование собственника:,Краснодарский край,"г.Сочи,с.Эсто-Садок,ул.Березовая д.132,т.43-90...",Телефон:,E-mail:,Адрес сайта:
4,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ОАО ""Черномортранснефть"" Сан-оздор.комплекс""Фр...",Тип:,Наименование собственника:,Краснодарский край,"г.Новороссийск,п.Абрау-Дюрсо, урочище Сухая ще...",Телефон:,E-mail:,Адрес сайта:


In [5]:
for i in range(5000, 10000):
    in_queue.put('http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/'+str(i))
    
jobs = []
for i in range(n_process):
    job = mp.Process(target=worker, args=(in_queue, out_queue, ))
    job.start()
    jobs.append(job)
    
for job in jobs:
    job.join()

while not out_queue.empty():
    result.loc[len(result)] = out_queue.get()

result.head()

list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/9798


,Url,Name,Type,Owner,Region,Adress,Phone,Email,Site
0,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"Отель ""Радужный"" ИП Дьяченко А.А.",Тип:,Наименование собственника:,Краснодарский край,"354008, Краснодасркий край, г. Сочи, ул. Пирог...",Телефон:,E-mail:,Адрес сайта:
1,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,Общество с ограниченной ответственностью «Сан...,"﻿ Дома отдыха, пансионаты и другие анало...",Общество с ограниченной ответственностью «Сан...,Краснодарский край,"354019, г. Сочи, Курортный проспект, д. 87",(8622) 67-16-70,frunze_sochi@mail.ru,http://www.frunze.net/
2,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ЗАО ""Нева"" Пансионат ""НЕВА ИНТЕРНЕЙШЕНЕЛ"" дире...",Тип:,Наименование собственника:,Краснодарский край,"г.Сочи , ул.Виноградная, 101 т.53-62-39,ф.53-5...",Телефон:,E-mail:,Адрес сайта:
3,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ООО""ВИЛ"",""Шале Поляна"" директор Шпалов Юрий Ми...",Тип:,Наименование собственника:,Краснодарский край,"г.Сочи,с.Эсто-Садок,ул.Березовая д.132,т.43-90...",Телефон:,E-mail:,Адрес сайта:
4,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ОАО ""Черномортранснефть"" Сан-оздор.комплекс""Фр...",Тип:,Наименование собственника:,Краснодарский край,"г.Новороссийск,п.Абрау-Дюрсо, урочище Сухая ще...",Телефон:,E-mail:,Адрес сайта:


In [6]:
for i in range(10000, 13000):
    in_queue.put('http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/'+str(i))
    
jobs = []
for i in range(n_process):
    job = mp.Process(target=worker, args=(in_queue, out_queue, ))
    job.start()
    jobs.append(job)
    
for job in jobs:
    job.join()

while not out_queue.empty():
    result.loc[len(result)] = out_queue.get()

result.head()

list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11420
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11598
list index out of range
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11599
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11600
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11597
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11601
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11604
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11603
list index out of range
http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1ai/index.php/displayAccommodation/11602
list index out of r

,Url,Name,Type,Owner,Region,Adress,Phone,Email,Site
0,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"Отель ""Радужный"" ИП Дьяченко А.А.",Тип:,Наименование собственника:,Краснодарский край,"354008, Краснодасркий край, г. Сочи, ул. Пирог...",Телефон:,E-mail:,Адрес сайта:
1,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,Общество с ограниченной ответственностью «Сан...,"﻿ Дома отдыха, пансионаты и другие анало...",Общество с ограниченной ответственностью «Сан...,Краснодарский край,"354019, г. Сочи, Курортный проспект, д. 87",(8622) 67-16-70,frunze_sochi@mail.ru,http://www.frunze.net/
2,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ЗАО ""Нева"" Пансионат ""НЕВА ИНТЕРНЕЙШЕНЕЛ"" дире...",Тип:,Наименование собственника:,Краснодарский край,"г.Сочи , ул.Виноградная, 101 т.53-62-39,ф.53-5...",Телефон:,E-mail:,Адрес сайта:
3,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ООО""ВИЛ"",""Шале Поляна"" директор Шпалов Юрий Ми...",Тип:,Наименование собственника:,Краснодарский край,"г.Сочи,с.Эсто-Садок,ул.Березовая д.132,т.43-90...",Телефон:,E-mail:,Адрес сайта:
4,http://xn----7sba3acabbldhv3chawrl5bzn.xn--p1a...,"ОАО ""Черномортранснефть"" Сан-оздор.комплекс""Фр...",Тип:,Наименование собственника:,Краснодарский край,"г.Новороссийск,п.Абрау-Дюрсо, урочище Сухая ще...",Телефон:,E-mail:,Адрес сайта:


In [7]:
writer = ExcelWriter('Parsing_glamping_res_v_12.xls')
result.to_excel(writer, sheet_name="minister")
writer.save()